# Reconstruct the SBML from a set of reactions

This is to test the FBA toolchain

In [1]:
import sys
import os
import copy
import PyFBA
import re

import inspect
inspect.getfile(PyFBA)

We are using /home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.0-py3.9.egg/PyFBA/Biochemistry/ModelSEEDDatabase for our data


'/home/redwards/GitHubsLinux/PyFBA/venv/lib/python3.9/site-packages/PyFBA-2.4-py3.9.egg/PyFBA/__init__.py'

# Read the Reactions, Enzymes, and Compounds Sets

This parses the ModelSeed data

In [2]:
modelseed = PyFBA.parse.model_seed.parse_model_seed_data('gramnegative')
print(f"There are {len(modelseed.compounds):,} compounds, {len(modelseed.reactions):,} reactions, and {len(modelseed.enzymes):,} enzymes in total")

There are 33,992 compounds, 43,774 reactions, and 9,423 enzymes in total


# Create a biomass equation

In [3]:
sbml_bm_eqn = '(0.00778132482043096) cpd00063: Ca2 (location: c) +  (0.352889948968272) cpd00156: L_Valine (location: e) +  (0.00778132482043096) cpd00030: Mn2 (location: e) +  (0.00778132482043096) cpd00205: K (location: c) +  (0.428732289454499) cpd00035: L_Alanine (location: e) +  (0.128039715997337) cpd00060: L_Methionine (location: e) +  (0.15480760087483) cpd00066: L_Phenylalanine (location: c) +  (0.00778132482043096) cpd00017: S_Adenosyl_L_methionine (location: c) +  (0.00778132482043096) cpd00010: CoA (location: c) +  (0.0609084652443221) cpd15665: Peptidoglycan_polymer_n_subunits (location: c) +  (0.0841036156544863) cpd00052: CTP (location: c) +  (0.00778132482043096) cpd10516: fe3 (location: e) +  (0.01468498342018) cpd00357: TTP (location: c) +  (0.00778132482043096) cpd00099: Cl_ (location: e) +  (0.01468498342018) cpd00356: dCTP (location: c) +  (0.00778132482043096) cpd10515: Fe2 (location: e) +  (0.00778132482043096) cpd00254: Mg (location: c) +  (0.242249358141304) cpd00322: L_Isoleucine (location: e) +  (0.00778132482043096) cpd00058: Cu2 (location: e) +  (0.00778132482043096) cpd00149: Co2 (location: c) +  (0.201205267995816) cpd00041: L_Aspartate (location: e) +  (1) cpd17043: RNA_transcription (location: c) +  (0.219496655995436) cpd00023: L_Glutamate (location: e) +  (0.219496655995436) cpd00053: L_Glutamine (location: e) +  (0.376088782528765) cpd00107: L_Leucine (location: e) +  (0.00778132482043096) cpd00220: Riboflavin (location: e) +  (0.179790960093822) cpd00054: L_Serine (location: e) +  (0.0472899299502361) cpd00065: L_Tryptophan (location: e) +  (0.0609084652443221) cpd02229: Bactoprenyl_diphosphate (location: c) +  (0.00778132482043096) cpd11493: ACP (location: c) +  (1) cpd17041: Protein_biosynthesis (location: c) +  (0.184698405654696) cpd00129: L_Proline (location: e) +  (0.135406821203723) cpd00038: GTP (location: c) +  (0.01468498342018) cpd00241: dGTP (location: c) +  (1) cpd17042: DNA_replication (location: c) +  (0.211466290532188) cpd00161: L_Threonine (location: e) +  (40.1101757365074) cpd00002: ATP (location: c) +  (0.00778132482043096) cpd00016: Pyridoxal_phosphate (location: c) +  (0.00778132482043096) cpd00048: Sulfate (location: e) +  (0.00778132482043096) cpd00003: NAD (location: c) +  (0.01468498342018) cpd00115: dATP (location: c) +  (0.115101904973216) cpd00069: L_Tyrosine (location: e) +  (0.00778132482043096) cpd00015: FAD (location: c) +  (0.201205267995816) cpd00132: L_Asparagine (location: e) +  (0.00778132482043096) cpd00006: NADP (location: c) +  (35.5386858537513) cpd00001: H2O (location: e) +  (0.0762884719008526) cpd00084: L_Cysteine (location: c) +  (0.0794113918032267) cpd00119: L_Histidine (location: e) +  (0.285970236774541) cpd00039: L_Lysine (location: e) +  (0.0908319049068452) cpd00062: UTP (location: c) +  (0.00778132482043096) cpd00034: Zn2 (location: e) +  (0.247156803702178) cpd00051: L_Arginine (location: e) +  (0.510820469745475) cpd00033: Glycine (location: e) >  (40) cpd00008: ADP (location: c) +  (39.9922186751796) cpd00009: Phosphate (location: e) +  (0.00778132482043096) cpd12370: apo_ACP (location: c) +  (1) cpd11416: Biomass (location: c) +  (40) cpd00067: H (location: e) +  (0.0609084652443221) cpd15666: Peptidoglycan_polymer_n_1_subunits (location: c) +  (0.405833094852252) cpd00012: PPi (location: e)'
sbml_left_compounds = {'cpd00066: L_Phenylalanine (location: c)' : 0.15480760087483,  'cpd00016: Pyridoxal_phosphate (location: c)' : 0.00778132482043096,  'cpd00132: L_Asparagine (location: e)' : 0.201205267995816,  'cpd00156: L_Valine (location: e)' : 0.352889948968272,  'cpd00099: Cl_ (location: e)' : 0.00778132482043096,  'cpd00038: GTP (location: c)' : 0.135406821203723,  'cpd00003: NAD (location: c)' : 0.00778132482043096,  'cpd17041: Protein_biosynthesis (location: c)' : 1.0,  'cpd00033: Glycine (location: e)' : 0.510820469745475,  'cpd00322: L_Isoleucine (location: e)' : 0.242249358141304,  'cpd00254: Mg (location: c)' : 0.00778132482043096,  'cpd17043: RNA_transcription (location: c)' : 1.0,  'cpd00048: Sulfate (location: e)' : 0.00778132482043096,  'cpd10515: Fe2 (location: e)' : 0.00778132482043096,  'cpd02229: Bactoprenyl_diphosphate (location: c)' : 0.0609084652443221,  'cpd11493: ACP (location: c)' : 0.00778132482043096,  'cpd00161: L_Threonine (location: e)' : 0.211466290532188,  'cpd00006: NADP (location: c)' : 0.00778132482043096,  'cpd00060: L_Methionine (location: e)' : 0.128039715997337,  'cpd00119: L_Histidine (location: e)' : 0.0794113918032267,  'cpd00052: CTP (location: c)' : 0.0841036156544863,  'cpd00051: L_Arginine (location: e)' : 0.247156803702178,  'cpd15665: Peptidoglycan_polymer_n_subunits (location: c)' : 0.0609084652443221,  'cpd00017: S_Adenosyl_L_methionine (location: c)' : 0.00778132482043096,  'cpd00030: Mn2 (location: e)' : 0.00778132482043096,  'cpd10516: fe3 (location: e)' : 0.00778132482043096,  'cpd00065: L_Tryptophan (location: e)' : 0.0472899299502361,  'cpd00084: L_Cysteine (location: c)' : 0.0762884719008526,  'cpd00023: L_Glutamate (location: e)' : 0.219496655995436,  'cpd17042: DNA_replication (location: c)' : 1.0,  'cpd00356: dCTP (location: c)' : 0.01468498342018,  'cpd00035: L_Alanine (location: e)' : 0.428732289454499,  'cpd00069: L_Tyrosine (location: e)' : 0.115101904973216,  'cpd00220: Riboflavin (location: e)' : 0.00778132482043096,  'cpd00129: L_Proline (location: e)' : 0.184698405654696,  'cpd00357: TTP (location: c)' : 0.01468498342018,  'cpd00205: K (location: c)' : 0.00778132482043096,  'cpd00149: Co2 (location: c)' : 0.00778132482043096,  'cpd00063: Ca2 (location: c)' : 0.00778132482043096,  'cpd00054: L_Serine (location: e)' : 0.179790960093822,  'cpd00001: H2O (location: e)' : 35.5386858537513,  'cpd00010: CoA (location: c)' : 0.00778132482043096,  'cpd00015: FAD (location: c)' : 0.00778132482043096,  'cpd00062: UTP (location: c)' : 0.0908319049068452,  'cpd00107: L_Leucine (location: e)' : 0.376088782528765,  'cpd00241: dGTP (location: c)' : 0.01468498342018,  'cpd00053: L_Glutamine (location: e)' : 0.219496655995436,  'cpd00039: L_Lysine (location: e)' : 0.285970236774541,  'cpd00034: Zn2 (location: e)' : 0.00778132482043096,  'cpd00058: Cu2 (location: e)' : 0.00778132482043096,  'cpd00002: ATP (location: c)' : 40.1101757365074,  'cpd00041: L_Aspartate (location: e)' : 0.201205267995816,  'cpd00115: dATP (location: c)' : 0.01468498342018}
sbml_right_compounds = {'cpd00067: H (location: e)' : 40.0,  'cpd00012: PPi (location: e)' : 0.405833094852252,  'cpd00008: ADP (location: c)' : 40.0,  'cpd11416: Biomass (location: c)' : 1.0,  'cpd12370: apo_ACP (location: c)' : 0.00778132482043096,  'cpd00009: Phosphate (location: e)' : 39.9922186751796,  'cpd15666: Peptidoglycan_polymer_n_1_subunits (location: c)' : 0.0609084652443221}
sbml_biomass = PyFBA.metabolism.Reaction('sbml_biomass', 'sbml_biomass')
sbml_biomass.equation = sbml_bm_eqn
parsecomp = re.compile('^(cpd\\d+): (.*?) \(location: (.)\)')
for c in sbml_left_compounds:
    m = parsecomp.match(c)
    if not m:
        sys.stderr.write(f"Can't parse {c}\n")
        
    cpd = modelseed.get_compound_by_id(m.group(1))
    if not cpd:
        cpd = modelseed.get_compound_by_name(m.group(1))    
    if not cpd:
        print(f"Left: {m.group(1)} not found")
        continue

    if cpd.name != m.group(2):
        sys.stderr.write(f"We found |{cpd.name}| for {m.group(1)} in the model seed, but have |{m.group(2)}| in the equation\n")
    newcomp = PyFBA.metabolism.CompoundWithLocation.from_compound(cpd, m.group(3))
    sbml_biomass.add_left_compounds({newcomp})
    sbml_biomass.set_left_compound_abundance(newcomp, sbml_left_compounds[c])
        

for c in sbml_right_compounds:
    m = parsecomp.match(c)
    if not m:
        sys.stderr.write(f"Can't parse {c}\n")

    cpd = modelseed.get_compound_by_id(m.group(1))
    if not cpd:
        cpd = modelseed.get_compound_by_name(m.group(1))    
    if not cpd:
        print(f"Left: {m.group(1)} not found")
        continue
        
    if cpd.name != m.group(2):
        sys.stderr.write(f"We found |{cpd.name}| for {m.group(1)} in the model seed, but have |{m.group(2)}| in the equation\n")
    newcomp = PyFBA.metabolism.CompoundWithLocation.from_compound(cpd, m.group(3))
    sbml_biomass.add_right_compounds({newcomp})
    sbml_biomass.set_right_compound_abundance(newcomp, sbml_right_compounds[c])


We found |L-Phenylalanine| for cpd00066 in the model seed, but have |L_Phenylalanine| in the equation
We found |Pyridoxal phosphate| for cpd00016 in the model seed, but have |Pyridoxal_phosphate| in the equation
We found |L-Asparagine| for cpd00132 in the model seed, but have |L_Asparagine| in the equation
We found |L-Valine| for cpd00156 in the model seed, but have |L_Valine| in the equation
We found |Cl-| for cpd00099 in the model seed, but have |Cl_| in the equation
We found |Protein biosynthesis| for cpd17041 in the model seed, but have |Protein_biosynthesis| in the equation
We found |L-Isoleucine| for cpd00322 in the model seed, but have |L_Isoleucine| in the equation
We found |RNA transcription| for cpd17043 in the model seed, but have |RNA_transcription| in the equation
We found |Fe+2| for cpd10515 in the model seed, but have |Fe2| in the equation
We found |Bactoprenyl diphosphate| for cpd02229 in the model seed, but have |Bactoprenyl_diphosphate| in the equation
We found |L-Thr

# Read the media and correct the names

In [4]:
# Read the media file
media = PyFBA.parse.read_media_file("/home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/Biochemistry/media/ArgonneLB.txt")
# Correct the names
media = PyFBA.parse.correct_media_names(media, modelseed.compounds)
print(f"The media has {len(media)} compounds")

Found media component cpd11595: chromate
Found media component cpd00007: O2
Found media component cpd00246: Inosine
Found media component cpd00069: L-Tyrosine
Found media component cpd00107: L-Leucine
Found media component cpd00119: L-Histidine
Found media component cpd00035: L-Alanine
Found media component cpd01048: Arsenate
Found media component cpd29996: Mg
Found media component cpd00033: Glycine
Found media component cpd00184: Thymidine
Found media component cpd26821: D-Glucose
Found media component cpd00091: UMP
Found media component cpd00067: H+
Found media component cpd00541: Lipoate
Found media component cpd00039: L-Lysine
Found media component cpd00018: AMP
Found media component cpd01012: Cd2+
Found media component cpd00220: Riboflavin
Found media component cpd00156: L-Valine
Found media component cpd00066: L-Phenylalanine
Found media component cpd00028: Heme
Found media component cpd00215: Pyridoxal
Found media component cpd03424: Vitamin B12
Found media component cpd00161: L

We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/PyFBA.2021-06-04T06:26:41.014029.log
Checking media compounds: Our compounds do not include  Fe2+
Checking media compounds: Our compounds do not include  fe3
Please note: The warnings about media not being found in compounds are not fatal.
It just means that we did not find that compound anywhere in the reactions, and so it is unlikely to be
needed or used. We typically see a few of these in rich media.


# Read the reactions

In [5]:
reactions_to_run = set()
with open('rgood.txt', 'r') as f:
    for l in f:
        if l.startswith('rxn'):
            reactions_to_run.add(l.strip())
print(f"There are {len(reactions_to_run)} reactions")

There are 1399 reactions


# Add the biomass equation to the reactions to run

In [6]:
#reactions_to_run.add(sbml_biomass)

In [7]:
for r in reactions_to_run:
    if r not in modelseed.reactions:
        sys.stderr.write(f"Error: reaction {r} not found\n")

# Run the FBA

In [8]:
uptake_secretion_reactions = set()

In [9]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, sbml_biomass,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


In parsing the media, cpd00220: Riboflavin is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00311: Guanosine is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00051: L-Arginine is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00001: H2O is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00971: Na+ is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00129: L-Proline is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00091: UMP is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00249: Uridine is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00009: Phosphate is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00119: L-Histidine is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing the media, cpd00007: O2 is a <class 'PyFBA.metabolism.compound.Compound'>
In parsing 

TypeError: Starting with v.2 reactions need PyFBA.metabolism.CompoundWithLocation objects not <class 'PyFBA.metabolism.compound.Compound'>